In [31]:
#import libraries and packages needed
import pandas as pd
import numpy as np
from pandas import read_csv
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [20]:
train = pd.read_csv("data/train.csv")

In [3]:
#analyzing the numerical variables of the training dataset
train.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [4]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
len(train)

614

In [6]:
train.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [8]:
train.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [9]:
train.groupby('Gender').size()

Gender
Female    112
Male      489
dtype: int64

## For simplicity sake lets take only rows that dont have any na

In [44]:
train_full = train.dropna()

In [45]:
len(train_full)

480

In [32]:
kfold = KFold(n_splits=10, random_state=7)

In [14]:
cate_str_cols = ['Gender', 'Married', 'Dependents', 'Education','Self_Employed','Property_Area','Loan_Status']

In [15]:
num_cols = ['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History']

In [41]:
X = train_full[num_cols]
y = train_full['Loan_Status']

In [48]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X,y)
results = cross_val_score(model, X, y, cv=kfold)

###### Mean accuracy of a model built on just numeric columns is 73%

In [49]:
print(results.mean())

0.7375


In [16]:
train_one_hot = train_full[num_cols]

In [17]:
lb = preprocessing.LabelBinarizer()
new_cols = num_cols
for col in cate_str_cols:
    col = train_full[col].tolist()
    X = pd.Series(list(lb.fit_transform(col)))
    train_one_hot =pd.concat([train_one_hot,X],axis = 1)
#     new_cols = new_cols.append(lb.classes_)

In [18]:
train_one_hot.columns

Index([  'ApplicantIncome', 'CoapplicantIncome',        'LoanAmount',
        'Loan_Amount_Term',    'Credit_History',                   0,
                         0,                   0,                   0,
                         0,                   0,                   0],
      dtype='object')

In [47]:
train_np.shape

(480, 17)

In [10]:
# model.fit(X,y)

In [29]:
X_test = test_full[num_cols]

KeyError: 'Loan_Status'

### Section to score the test dataset

In [86]:
test_dropna = read_csv("data/test.csv").dropna()

In [91]:
test_short = test_dropna[num_cols]

In [104]:
out = pd.Series(model.predict(test_short),name='Loan_Status')

In [105]:
test_dropna = test_dropna.reset_index()

In [106]:
test_dropna.head()

,level_0,index,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,0,0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,1,1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,2,2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,3,4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban
4,4,5,LP001054,Male,Yes,0,Not Graduate,Yes,2165,3422,152.0,360.0,1.0,Urban


In [109]:
output = pd.concat([test_dropna['Loan_ID'],out],axis=1)

In [ ]:
.to_csv('sample_submissions.csv',index=False)